### **SATCAT Data Pipeline: Global Registry Standardization**

**Dataset:** CelesTrak Satellite Catalog (SATCAT)
**Objective:** Transform raw orbital tracking data into a physics-reconstructed, "Gold Standard" global registry.

### **The Engineering Challenge**
While the SATCAT is the premier source for orbital tracking (location), it presents a significant **Physics Transparency Gap**. While it identifies ~60,000 objects, it is effectively **"Mass-Blind"** out of the box.

1.  **Ingestion & Schema Alignment:** Normalize headers to snake_case and implement in_orbit logic. 
2.  **Universal Numeric Sanitization:** Neutralize 0.0 placeholders and enforce strict type-safety.
3.  **Keplerian Density Engineering:** Mathematically derive missing orbital periods using Kepler’s Third Law.
4.  **High-Fidelity Enrichment:** Synchronize with ucs_cleaned.csv to inject precision mass data for the active fleet.
5.  **Tiered Mass Imputation:** Apply ESA-standard proxies for the "Invisible Population" (Debris/Rocket Bodies).
6.  **Categorical Hardening:** Standardize object types and operational status codes for relational integrity.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

### **Stage 1: Ingestion & Schema Alignment**
**The Problem:** The raw SATCAT headers use legacy uppercase formatting (e.g., `NORAD_CAT_ID`), which is inconsistent with our UCS snake_case convention. Furthermore, a "Gold Standard" dataset must preserve all original metadata to ensure no information is lost during the standardization process.

**The Solution:** * **Global Renaming:** We map critical physical and relational headers to `snake_case` to match the UCS pipeline's "DNA".
* **Feature Preservation:** We retain 100% of the original columns, only renaming the essential ones for programmatic efficiency.
* **In-Orbit Logic:** We introduce the `in_orbit` boolean—a primary feature that separates active kinetic threats from historical decay records.

In [2]:
raw_registry = pd.read_csv('../data/original/satcat.csv')

rename_map = {
    'OBJECT_NAME': 'object_name',
    'OBJECT_ID': 'cospar_id',            # Perfect Match with UCS
    'NORAD_CAT_ID': 'norad_id',          # Perfect Match with UCS
    'OBJECT_TYPE': 'object_type',        # Refined in Stage 4
    'OPS_STATUS_CODE': 'ops_status',   
    'OWNER': 'owner_code',               # To be enriched by UCS 'owner'
    'LAUNCH_DATE': 'launch_date',        # Standardized to datetime
    'LAUNCH_SITE': 'launch_site',        # Matches UCS
    'DECAY_DATE': 'decay_date',
    'PERIOD': 'period_minutes',          # Standardized Physics
    'INCLINATION': 'inclination_degrees',# Standardized Physics
    'APOGEE': 'apogee_km',               # Standardized Physics
    'PERIGEE': 'perigee_km',             # Standardized Physics
    'RCS': 'rcs',                        # Core for Kinetic Modeling
    'DATA_STATUS_CODE': 'data_status', 
    'ORBIT_CENTER': 'orbit_center',    
    'ORBIT_TYPE': 'orbit_type_code'      # Changed to prevent merge collision
}

# batch rename of all columns to standardized names
# that align with UCS where at all possible
registry = raw_registry.rename(columns=rename_map).copy()

# sets registry['in_orbit'] to 1 if registry['decay_date'] is NaN, sets registry['in_orbit'] to 0 otherwise
# this allows us to quickly filter for objects that are still in orbit without having to check for NaN values each time
registry['in_orbit'] = registry['decay_date'].isnull().astype(int)

print(f"--- Stage 1 Audit ---")
print(f"Total Columns Retained: {len(registry.columns)}")
print(f"Total Records:         {len(registry):,}")
print(f"In-Orbit Boolean:      Added (Density: {registry['in_orbit'].mean():.1%})")

--- Stage 1 Audit ---
Total Columns Retained: 18
Total Records:         67,464
In-Orbit Boolean:      Added (Density: 48.7%)


### **Stage 1.1: Strategic Ghost Column Audit**
**The Problem:** Raw CSV exports often contain "Ghost Columns" — unpopulated placeholders created by formatting artifacts or trailing delimiters in the original database. These columns inflate memory usage and create "Wide Data" noise without adding informational value.

**The Solution:** We implement a **Dynamic Artifact Filter** to identify and purge any columns matching the `Unnamed` pattern or those containing 100% null values. This ensures the dataset remains lean and focused on valid orbital attributes while preserving 100% of the legitimate SATCAT metadata.

In [3]:
# Filter off unwanted columns dynamically to account for version updates

# there are multiple columns that are named 'Unnamed: X'
# this will dynamically find all such columns
unnamed_columns = [col for col in registry.columns if 'Unnamed' in col]

# there are multiple columns that are 100% null
# this will dynamically find all such columns
null_columns = [col for col in registry.columns if registry[col].isnull().all()]

# combine both lists to get all of the 'ghost' columns
ghost_columns = list(set(unnamed_columns + null_columns))

print("--- Ghost Column Audit ---")
if ghost_columns:
    # if there are any columns in the ghost_columns list, drop them from the registry dataframe
    print(f"Found {len(ghost_columns)} artifact columns: {ghost_columns}")
    registry.drop(columns=ghost_columns, inplace=True)
    print(f"✅ SUCCESS: Artifacts purged. Remaining columns: {len(registry.columns)}")
else:
    print("✅ CLEAN: No ghost columns or 100% null artifacts detected.")

# a list of essential keys that MUST be present in the cleaned registry
essential_keys = ['norad_id', 'object_name', 'in_orbit', 'period_minutes', 'perigee_km', 'apogee_km', 'inclination_degrees', 'rcs']

# verify that all essential keys are present in the cleaned registry
integrity_check = all(key in registry.columns for key in essential_keys)

print(f"Relational Integrity Check: {'PASSED' if integrity_check else 'FAILED'}")

--- Ghost Column Audit ---
✅ CLEAN: No ghost columns or 100% null artifacts detected.
Relational Integrity Check: PASSED


### **Stage 1.2: Pre-Sanitization Health Audit**
**The Problem:** Before enforcing physics standardization, we must quantify the scale of the **Transparency Gap** in the raw registry. Treating `0.0` or string artifacts as valid data during initial EDA would lead to a "Ghost Population" that appears to have no mass or motion, skewing our baseline risk assessments.

**The Solution:** We execute a technical audit to identify "Non-Physical" values across our five core attributes. This report tracks:
* **String Artifacts:** Non-numeric entries that force columns into `object` types.
* **Placeholder Zeros:** Valid numeric `0.0` entries that actually represent missing data in the SATCAT.
* **Density Baseline:** The true percentage of "Physics-Ready" data available in the raw source.

In [4]:
# kinetic and geometric attributes
physics_cols = ['period_minutes', 'inclination_degrees', 'apogee_km', 'perigee_km', 'rcs']

print(f"{'--- RAW DATA HEALTH AUDIT ---':^55}")
print(f"{'Column':<20} | {'Non-Numeric':<12} | {'Zeros':<8} | {'Health %'}")
print("-" * 55)

total_records = len(registry)

for col in physics_cols:
    if col in registry.columns:
        # Count Non-Numeric (Strings/NaNs)
        # We try to convert to numeric safely to find what is currently a string
        numeric_series = pd.to_numeric(registry[col], errors='coerce')

        # step through the columns, check NaNs for each column and determine how many rows are NaN or a string
        non_numeric = registry[col].isna().sum() + (registry[col].apply(lambda x: isinstance(x, str))).sum()
        
        # Count Placeholder Zeros (Numeric 0.0)
        # This lets us count how many valid numeric entries are actually zero
        # by using the numeric_series we created above and summing where it equals zero
        zeros = (numeric_series == 0).sum()
        
        # Calculate "Physics-Ready" Density
        # Valid = Not NaN AND Not Zero
        valid_count = total_records - (non_numeric + zeros)
        health_pct = (valid_count / total_records) * 100
        
        print(f"{col:<20} | {non_numeric:>12,} | {zeros:>8,} | {health_pct:>7.1f}%")

print("-" * 55)
print(f"Total Registry Records: {total_records:,}")
print("Note: 'Health %' represents records that are both numeric and non-zero.")

             --- RAW DATA HEALTH AUDIT ---             
Column               | Non-Numeric  | Zeros    | Health %
-------------------------------------------------------
period_minutes       |          930 |       26 |    98.6%
inclination_degrees  |          930 |       34 |    98.6%
apogee_km            |          930 |       27 |    98.6%
perigee_km           |          930 |       38 |    98.6%
rcs                  |       34,533 |        0 |    48.8%
-------------------------------------------------------
Total Registry Records: 67,464
Note: 'Health %' represents records that are both numeric and non-zero.


### **Stage 2: Universal Numeric & Identifier Sanitization**
**The Problem:** High-fidelity orbital modeling requires strict numeric types. However, fields like `rcs`, `period_minutes`, and `perigee_km` often contain `0.0` as a placeholder for "Unknown" data. Furthermore, primary identifiers like `norad_id` and `cospar_id` can contain decimal artifacts or hidden whitespace that prevent clean merging.

**The Solution:**
* **Numeric Coercion:** We apply `pd.to_numeric` across all core physics columns to ensure a stable `float64` foundation.
* **Placeholder Neutralization:** We convert `0.0` values to `NaN` to ensure missing data is handled correctly by statistical functions.
* **ID Normalization:** We force `norad_id` to a clean integer-string and apply a deep string-scrub to `cospar_id` to ensure 1:1 join-readiness with the UCS dataset.

In [5]:
# Expanded lists for universal sanitization
physics_cols = ['period_minutes', 'inclination_degrees', 'apogee_km', 'perigee_km', 'rcs']
id_numeric_cols = ['norad_id']
id_string_cols = ['cospar_id']

print(f"Sanitizing {len(physics_cols) + len(id_numeric_cols) + len(id_string_cols)} attributes...")

# Standardize Numeric Identifiers
for col in id_numeric_cols:
    if col in registry.columns:
        # Strip decimal artifacts (e.g., 25544.0 -> 25544)
        registry[col] = pd.to_numeric(registry[col], errors='coerce')
        registry = registry.dropna(subset=[col])
        registry[col] = registry[col].astype(int).astype(str)

# Standardize String Identifiers
for col in id_string_cols:
    if col in registry.columns:
        # Strip whitespace and normalize case to prevent "Invisible Bug" merge failures
        registry[col] = registry[col].astype(str).str.strip().str.upper()
        # Neutralize 'NAN' strings
        registry[col] = registry[col].replace('NAN', np.nan)

# Sanitization Loop for Physics (UCS Stage 3.2 & 4.2 Logic)
for col in physics_cols:
    if col in registry.columns:
        # Force to numeric foundation
        registry[col] = pd.to_numeric(registry[col], errors='coerce')
        
        # Neutralize 0.0 placeholders representing 'Missing' data
        registry[col] = registry[col].replace(0, np.nan)
        
        # Enforce Physical Constraints: Geometry cannot be negative
        if col != 'rcs':
            registry.loc[registry[col] < 0, col] = 0

# Verification Diagnostic
print("\n--- Final Sanitization Health Check ---")
for col in physics_cols + id_numeric_cols + id_string_cols:
    null_count = registry[col].isnull().sum()
    dtype = registry[col].dtype
    print(f"{col:<20} | Nulls: {null_count:>6,} | Type: {dtype}")

print("\n✅ Stage 2 Complete: Numeric and Identifier integrity enforced.")

Sanitizing 7 attributes...

--- Final Sanitization Health Check ---
period_minutes       | Nulls:    956 | Type: float64
inclination_degrees  | Nulls:    964 | Type: float64
apogee_km            | Nulls:    957 | Type: float64
perigee_km           | Nulls:    968 | Type: float64
rcs                  | Nulls: 34,533 | Type: float64
norad_id             | Nulls:      0 | Type: object
cospar_id            | Nulls:      0 | Type: object

✅ Stage 2 Complete: Numeric and Identifier integrity enforced.


### **Stage 2.1: High-Fidelity Geometry & Mass Enrichment (UCS Sync)**
**The Problem:** Stage 2 identified core orbital elements and a total absence of mass data. While these can be mathematically derived in Stage 3, utilizing verified ground-truth observations from the verified UCS registry provides a higher level of "Gold Standard" precision for the active fleet.

**The Solution:** We synchronize the SATCAT with `ucs_cleaned.csv` immediately after sanitization. This creates a "Hierarchy of Truth" where verified observations from the active satellite registry override SATCAT nulls, repairing the catalog's active population before we resort to secondary Keplerian derivations or statistical proxies.

In [6]:
ucs_path = '../data/clean/ucs_cleaned.csv'
ucs_clean = pd.read_csv(ucs_path, dtype={'norad_id': str})
 
repair_cols = [
    'norad_id', 'launch_mass_kg', 'period_minutes', 
    'inclination_degrees', 'apogee_km', 'perigee_km', 'eccentricity'
]

print(f"Synchronizing SATCAT with UCS high-fidelity data...")

# left sided merge
# append UCS columns with '_ucs' suffix to avoid overwriting existing data unintentionally
registry = registry.merge(ucs_clean[repair_cols], on='norad_id', how='left', suffixes=('', '_ucs'))

# The Enrichment & Repair Audit
enriched_fields = ['period_minutes', 'inclination_degrees', 'apogee_km', 'perigee_km', 'eccentricity', 'launch_mass_kg']

print(f"\n{'--- RELATIONAL REPAIR AUDIT ---':^55}")
print(f"{'Attribute':<20} | {'Repaired':<10} | {'Final Nulls'}")
print("-" * 55)

for col in enriched_fields:
    ucs_col = f"{col}_ucs" if f"{col}_ucs" in registry.columns else col
    if ucs_col != col:
        pre_nulls = registry[col].isnull().sum()
        registry[col] = registry[col].fillna(registry[ucs_col])
        post_nulls = registry[col].isnull().sum()
        repaired = pre_nulls - post_nulls
        print(f"{col:<20} | {repaired:>10,} | {post_nulls:>11,}")

# Clean up temporary UCS columns
cols_to_drop = [c for c in registry.columns if c.endswith('_ucs')]
registry.drop(columns=cols_to_drop, inplace=True)

# Final Sanitization Health Check (Verbose & Pretty)
print(f"\n{'--- FINAL STAGE 2.1 HEALTH GATE ---':^55}")
print(f"{'Feature':<20} | {'Type':<10} | {'Density'}")
print("-" * 55)

# Include identifiers and sanitized physics for the final gate
gate_cols = ['norad_id', 'cospar_id', 'period_minutes', 'inclination_degrees', 'apogee_km', 'perigee_km', 'launch_mass_kg']

for col in gate_cols:
    null_count = registry[col].isnull().sum()
    density = (1 - (null_count / len(registry))) * 100
    dtype = str(registry[col].dtype)
    print(f"{col:<20} | {dtype:<10} | {density:>8.1f}%")

print("-" * 55)
print(f"✅ Stage 2.1 Complete: High-fidelity active fleet data synchronized.")

Synchronizing SATCAT with UCS high-fidelity data...

            --- RELATIONAL REPAIR AUDIT ---            
Attribute            | Repaired   | Final Nulls
-------------------------------------------------------
period_minutes       |         22 |         934
inclination_degrees  |         26 |         938
apogee_km            |         23 |         934
perigee_km           |         25 |         943

          --- FINAL STAGE 2.1 HEALTH GATE ---          
Feature              | Type       | Density
-------------------------------------------------------
norad_id             | object     |    100.0%
cospar_id            | object     |    100.0%
period_minutes       | float64    |     98.6%
inclination_degrees  | float64    |     98.6%
apogee_km            | float64    |     98.6%
perigee_km           | float64    |     98.6%
launch_mass_kg       | float64    |     11.2%
-------------------------------------------------------
✅ Stage 2.1 Complete: High-fidelity active fleet data synchr

### **Stage 2.2: Timeline Reconstruction**
**The Problem:** Standard integer types in pandas cannot store `NaN` values, often forcing years and ages into floating-point formats (e.g., `2026.0`). This creates "Type Integrity" errors during final audits and inconsistent data "smell" when compared to the UCS registry.

**The Solution:** We implement the **nullable integer (`Int64`)** specification. This ensures that `launch_year` and `sat_age_years` are stored as pure integers while preserving the ability to handle missing data without defaulting to floats.

In [7]:
print("Executing Timeline Reconstruction...")

# Standardize Dates
registry['launch_date'] = pd.to_datetime(registry['launch_date'], errors='coerce')
registry['decay_date'] = pd.to_datetime(registry['decay_date'], errors='coerce')

# Derive Timeline Attributes
registry['launch_year'] = registry['launch_date'].dt.year
registry['sat_age_years'] = 2026 - registry['launch_year']

# Enforce Nullable Integer Types (The "Gold Standard" Fix)
# 'Int64' (Capital I) is the pandas-specific type for integers that allow NaNs
registry['launch_year'] = registry['launch_year'].astype('Int64')
registry['sat_age_years'] = registry['sat_age_years'].astype('Int64')

in_orbit_mask = registry['in_orbit'] == 1
avg_age = registry[in_orbit_mask]['sat_age_years'].mean()
print(f"✅ Stage 2.2 Complete: Timeline hardened. Average In-Orbit Age: {avg_age:.1f} years.")

Executing Timeline Reconstruction...
✅ Stage 2.2 Complete: Timeline hardened. Average In-Orbit Age: 19.7 years.


### **Stage 3: Keplerian Reconstruction (The Density Engine)**
**The Problem:** Despite being a tracking catalog, many entries possess Perigee and Apogee data but are missing a recorded `period_minutes`. To achieve 100% density for our kinetic models, we cannot rely on incomplete records or "ghost" orbits that appear to have no motion.

**The Solution:** We implement **Kepler’s Third Law**. By treating Earth's gravitational constant ($\mu$) and radius as constants, we can mathematically derive the missing periods from the existing orbital geometry. This ensures every object flagged as `in_orbit` is physics-ready for downstream kinetic energy calculations.

In [8]:
# Earth Constants for Orbital Mechanics
earth_radius = 6378.137
mu = 398600.4418 # km^3/s^2 (Earth's gravitational parameter)

def calculate_kepler_period(row):
    """
    Derives orbital period from altitudes if the period is missing.
    Matches the derivation logic used in ucs_cleanup Stage 4.2.
    """
    # Only derive if Period is missing but we have altitudes
    if pd.isna(row['period_minutes']) and not pd.isna(row['perigee_km']) and not pd.isna(row['apogee_km']):
        # a = semi-major axis (Earth Radius + Average Altitude)
        a = earth_radius + ((row['perigee_km'] + row['apogee_km']) / 2)
        # T = 2 * pi * sqrt(a^3 / mu)
        period_seconds = 2 * np.pi * np.sqrt(a**3 / mu)
        return period_seconds / 60
    return row['period_minutes']

print("Executing Keplerian Reconstruction...")
registry['period_minutes'] = registry.apply(calculate_kepler_period, axis=1)

# Diagnostic Audit of the In-Orbit Population
in_orbit_mask = registry['in_orbit'] == 1
missing_p = registry[in_orbit_mask]['period_minutes'].isnull().sum()

print(f"\n--- Keplerian Audit ---")
print(f"Remaining Missing Periods (In-Orbit): {missing_p}")
print(f"Period Density (In-Orbit):           {registry[in_orbit_mask]['period_minutes'].notna().mean():.1%}")

if missing_p == 0:
    print("\n🚀 SUCCESS: 100% Period density achieved for the in-orbit population.")

Executing Keplerian Reconstruction...

--- Keplerian Audit ---
Remaining Missing Periods (In-Orbit): 604
Period Density (In-Orbit):           98.2%


### **Stage 3.1: Orbital Classification & Final Period Imputation**
**The Problem:** Our Keplerian reconstruction achieved 98.1% density, but 615 objects remain "Physics-Blind" because they lack both a recorded period and altitude data. In the UCS pipeline, we addressed similar gaps by leveraging **Grouped Median Imputation**.

**The Solution:** * **Regime Classification:** We implement a `classify_orbit` function to group all objects into **LEO, MEO, GEO,** or **High Elliptical** based on their orbital period.
* **Peer-Group Imputation:** For the final 615 objects, we fill the missing `period_minutes` using the median value of their respective orbital class. This mirrors the Stage 4.2 logic from the UCS cleanup and ensures 100% density for the in-orbit population.

In [9]:
def classify_orbit(period):
    """
    Translates orbital period into standardized regimes.
    """
    if pd.isnull(period) or period <= 0:
        return 'UNKNOWN'
    elif period < 128:
        return 'LEO'
    elif 1400 <= period <= 1460:
        return 'GEO'
    elif 128 <= period < 1400:
        return 'MEO'
    else:
        return 'Elliptical'
    
# Apply initial orbit classification across the entire registry
registry['orbit_class'] = registry['period_minutes'].apply(classify_orbit)

# We use the median of the orbit_class to fill the final gaps
print(f"Imputing final {registry[registry['in_orbit'] == 1]['period_minutes'].isnull().sum()} periods...")

# groupby orbit_class and transform to get the median period for each orbit class
orbit_medians = registry.groupby('orbit_class')['period_minutes'].transform('median')
registry['period_minutes'] = registry['period_minutes'].fillna(orbit_medians)

# Global Safety Net (In case orbit_class was 'UNKNOWN')
# use the overall median to fill the last of the gaps
global_median = registry['period_minutes'].median()
registry['period_minutes'] = registry['period_minutes'].fillna(global_median)

in_orbit_mask = registry['in_orbit'] == 1

# filter by in-orbit and count remaining missing periods
missing_final = registry[in_orbit_mask]['period_minutes'].isnull().sum()

print(f"\n--- Final Physics Density Audit ---")
print(f"Remaining Missing Periods (In-Orbit): {missing_final}")
print(f"Final Period Density:                {registry[in_orbit_mask]['period_minutes'].notna().mean():.1%}")

if missing_final == 0:
    print("\n🚀 SUCCESS: 100% Physics density achieved via Grouped Median Imputation.")

Imputing final 604 periods...

--- Final Physics Density Audit ---
Remaining Missing Periods (In-Orbit): 0
Final Period Density:                100.0%

🚀 SUCCESS: 100% Physics density achieved via Grouped Median Imputation.


### **Stage 3.2: Final Geometric Sweep**
**The Problem:** While our Period density is now at 100%, the underlying geometric attributes—`inclination_degrees`, `apogee_km`, and `perigee_km`—still contain the `NaN` values we neutralized in Stage 2. A "Gold Standard" dataset requires 100% density across the entire physical profile to support precise kinetic modeling.

**The Solution:** Perform a final median sweep across the remaining geometric features. We leverage the `orbit_class` we just engineered to fill these gaps with regime-specific medians, ensuring that an "Unknown LEO" object receives the physical characteristics typical of its neighbors.

In [10]:
sweep_cols = ['inclination_degrees', 'apogee_km', 'perigee_km']

print("Executing Final Geometric Sweep...")

for col in sweep_cols:
    if col in registry.columns:
        # Primary Fill: Grouped by the Orbit Class we just created
        regime_medians = registry.groupby('orbit_class')[col].transform('median')
        registry[col] = registry[col].fillna(regime_medians)
        
        # Safety Fill: Global Median (for any 'UNKNOWN' regimes)
        registry[col] = registry[col].fillna(registry[col].median())

print(f"\n{'--- FINAL GEOMETRY AUDIT ---':^45}")
print(f"{'Feature':<25} | {'Completeness'}")
print("-" * 45)

for col in ['period_minutes'] + sweep_cols:
    coverage = registry[registry['in_orbit'] == 1][col].notna().mean()
    print(f"{col:<25} | {coverage:>12.1%}")

print("-" * 45)
print("🚀 PASS: 100% Geometric density achieved for the in-orbit population.")

Executing Final Geometric Sweep...

        --- FINAL GEOMETRY AUDIT ---         
Feature                   | Completeness
---------------------------------------------
period_minutes            |       100.0%
inclination_degrees       |       100.0%
apogee_km                 |       100.0%
perigee_km                |       100.0%
---------------------------------------------
🚀 PASS: 100% Geometric density achieved for the in-orbit population.


### **Stage 3.3: Integrity Polish & Eccentricity Derivation**
**The Problem:** While our altitude and period density are at 100%, the SATCAT is missing an explicit **`eccentricity`** field, which is a core requirement for the kinetic models used in the UCS pipeline. Additionally, **`geo_longitude`** must be neutralized to ensure it doesn't contain string artifacts.

**The Solution:** 
* **Mathematical Derivation:** We derive eccentricity using the formula $e = (r_a - r_p) / (r_a + r_p)$, where $r$ is the distance from the Earth's center. 
* **Schema Alignment:** We add these features to our final "Gold Standard" sweep to ensure 100% parity with the UCS dataset.

In [11]:
# AI helped me find the formula but the function itself is not complicated.
# It is 2 basic math operations and then we return a division result.
# but I didn't know the formula.
#
# ra = apogee distance from Earth's center
# rp = perigee distance from Earth's center
# Derive Eccentricity
# Formula: e = (r_a - r_p) / (r_a + r_p)
# where r_a = apogee_km + Earth_radius, r_p = perigee_km + Earth_radius

earth_radius = 6378.137

def derive_eccentricity(row):
    ra = row['apogee_km'] + earth_radius  # Distance from Earth center to apogee
    rp = row['perigee_km'] + earth_radius  # Distance from Earth center to perigee
    return (ra - rp) / (ra + rp)

print("Deriving orbital eccentricity...")
registry['eccentricity'] = registry.apply(derive_eccentricity, axis=1)

# Initialize geo_longitude if it doesn't exist (to match UCS schema)
if 'geo_longitude' not in registry.columns:
    registry['geo_longitude'] = np.nan

# Final Sweep for the "Missing Two"
final_sweep = ['eccentricity', 'geo_longitude']

for col in final_sweep:
    # Grouped Median Fill
    regime_medians = registry.groupby('orbit_class')[col].transform('median')
    registry[col] = registry[col].fillna(regime_medians)
    
    # Global Safety Net
    # GEO longitude defaults to 0 for non-GEO
    registry[col] = registry[col].fillna(0.0)

print(f"\n{'--- COMPLETE GEOMETRIC AUDIT ---':^45}")
print(f"{'Feature':<25} | {'Completeness'}")
print("-" * 45)

all_geo = ['period_minutes', 'inclination_degrees', 'apogee_km', 'perigee_km', 'eccentricity', 'geo_longitude']
for col in all_geo:
    coverage = registry[registry['in_orbit'] == 1][col].notna().mean()
    print(f"{col:<25} | {coverage:>12.1%}")

print("-" * 45)
print("🚀 PASS: 100% Parity with UCS geometric schema achieved.")

Deriving orbital eccentricity...

      --- COMPLETE GEOMETRIC AUDIT ---       
Feature                   | Completeness
---------------------------------------------
period_minutes            |       100.0%
inclination_degrees       |       100.0%
apogee_km                 |       100.0%
perigee_km                |       100.0%
eccentricity              |       100.0%
geo_longitude             |       100.0%
---------------------------------------------
🚀 PASS: 100% Parity with UCS geometric schema achieved.


### **Stage 4: Categorical Hardening & String Scrubbing**
**The Problem:** The raw SATCAT utilizes abbreviated codes (e.g., `PAY`, `R/B`) and inconsistent string formatting. Mixed-case entries and trailing whitespaces—"Invisible Bugs"—can cause silent failures during categorical grouping or when merging with the UCS dataset later in the pipeline.

**The Solution:**
* **Object Mapping:** We translate raw codes into a controlled vocabulary (`PAYLOAD`, `ROCKET BODY`, `DEBRIS`) to ensure clarity.
* **Geopolitical Normalization:** We enforce a strict uppercase and `strip()` operation on `owner_code` and `launch_site` to ensure unique labels for geopolitical analysis.
* **Deep Scrub:** We iterate through all text-based columns to neutralize whitespace artifacts and ensure the registry meets our "Gold Standard" for data cleanliness.

In [12]:
# Map Object Types to human-readable vocabulary
type_map = {
    'PAY': 'PAYLOAD', 
    'R/B': 'ROCKET BODY', 
    'DEB': 'DEBRIS',
    'UNK': 'UNKNOWN'
}

print("Standardizing object classifications...")
registry['object_type'] = registry['object_type'].str.strip().str.upper().map(type_map).fillna('UNKNOWN')

# Global String Scrubbing
text_cols = registry.select_dtypes(include=['object']).columns

print(f"Executing Deep Scrub on {len(text_cols)} text columns...")
for col in text_cols:
    registry[col] = registry[col].astype(str).str.strip().str.upper()

# Neutralize 'NAN' strings back to proper np.nan
registry = registry.replace('NAN', np.nan)

print("\n--- Categorical Distribution ---")
print(registry['object_type'].value_counts())

print(f"\n✅ Stage 4 Complete: {len(text_cols)} columns standardized and scrubbed.")

Standardizing object classifications...
Executing Deep Scrub on 11 text columns...

--- Categorical Distribution ---
object_type
DEBRIS         35739
PAYLOAD        24763
ROCKET BODY     6807
UNKNOWN          155
Name: count, dtype: int64

✅ Stage 4 Complete: 11 columns standardized and scrubbed.


### **Stage 4.1: Operational Status Hardening**
**The Problem:** Raw tracking data utilizes legacy shorthand for satellite health (e.g., `+` for operational, `-` for non-operational). These codes are insufficient for high-level risk reporting or geopolitical audits.

**The Solution:** We map the single-character status codes to standardized, human-readable labels. This ensures that the registry's operational context is immediately accessible and ready for "Zombie" identification in the next phase of the project.

In [13]:
# Map legacy single-character status codes to human-readable labels
# Source: CelesTrak SATCAT Legend
status_map = {
    '+': 'OPERATIONAL',
    '-': 'NON-OPERATIONAL',
    'P': 'PARTIAL',
    'B': 'BACKUP/STANDBY',
    'S': 'STANDBY',
    'X': 'EXTENDED MISSION',
    'D': 'DECAYED'
}

print("Hardening operational status codes...")

# Apply mapping and handle missing values
registry['ops_status'] = registry['ops_status'].map(status_map).fillna('UNKNOWN')

print("\n--- Operational Status Distribution ---")
print(registry['ops_status'].value_counts())

print("\n✅ Stage 4.1 Complete: Legacy status codes standardized.")

Hardening operational status codes...

--- Operational Status Distribution ---
ops_status
DECAYED             34626
UNKNOWN             16654
OPERATIONAL         14287
NON-OPERATIONAL      1453
PARTIAL               399
BACKUP/STANDBY         23
EXTENDED MISSION       16
STANDBY                 6
Name: count, dtype: int64

✅ Stage 4.1 Complete: Legacy status codes standardized.


### **Stage 4.2: Physical Binning (RCS Size Class)**
**The Problem:** While we possess numeric Radar Cross Section (RCS) data, high-level auditing and visualization are often more effective when using industry-standard size categories.

**The Solution:** We mathematically bin the numeric `rcs` into standard classes: **SMALL** (<0.1 $m^2$), **MEDIUM** (0.1-1.0 $m^2$), and **LARGE** (>1.0 $m^2$). This allows us to track "Size Distribution" in the orbital environment alongside raw mass.

In [14]:
# basic RCS size categorization
# 
def categorize_rcs(val):
    if pd.isna(val): return 'UNKNOWN'
    if val < 0.1:    return 'SMALL'
    if val < 1.0:    return 'MEDIUM'
    return 'LARGE'

print("Deriving size categories from numeric RCS...")
registry['rcs_class'] = registry['rcs'].apply(categorize_rcs)

print("\n--- RCS Size Distribution (Derived) ---")
print(registry['rcs_class'].value_counts())

print("\n✅ Stage 4.3 Complete: Physical size categories derived.")

Deriving size categories from numeric RCS...

--- RCS Size Distribution (Derived) ---
rcs_class
UNKNOWN    34533
SMALL      18496
LARGE       8515
MEDIUM      5920
Name: count, dtype: int64

✅ Stage 4.3 Complete: Physical size categories derived.


### **Stage 4.3: Geopolitical "Owner" Transparency Audit**
**The Problem:** Raw tracking catalogs often contain geopolitical artifacts such as trailing whitespaces or inconsistent country codes (e.g., 'US' vs 'USA'). These inconsistencies prevent accurate reporting of "Kinetic Responsibility"—which nation owns the most mass/risk in a given orbital regime.

**The Solution:** We perform a final Geopolitical Audit to verify that the **Stage 4 Deep Scrub** successfully unified the `owner_code` field. This report identifies the top \"Orbital Stakeholders\" and ensures the categorical data is 100% prepared for relational merging with the UCS registry.

In [15]:
in_orbit_mask = registry['in_orbit'] == 1
top_owners = registry[in_orbit_mask]['owner_code'].value_counts()

print(f"{'--- GEOPOLITICAL STAKEHOLDER AUDIT ---':^55}")
print(f"{'Rank':<5} | {'Code':<10} | {'Object Count':<15} | {'Share'}")
print("-" * 55)

for i, (code, count) in enumerate(top_owners.head(10).items(), 1):
    share = (count / in_orbit_mask.sum()) * 100
    print(f"{i:<5} | {str(code):<10} | {count:>12,} | {share:>6.1f}%")

unique_codes = registry['owner_code'].dropna().unique()
dirty_artifacts = [c for c in unique_codes if str(c) != str(c).strip().upper()]

print("-" * 55)
if not dirty_artifacts:
    print(f"✅ PASS: {len(unique_codes)} unique owner codes verified as standardized.")
else:
    print(f"❌ FAIL: {len(dirty_artifacts)} dirty artifacts detected. Re-run Stage 4 scrub.")

        --- GEOPOLITICAL STAKEHOLDER AUDIT ---         
Rank  | Code       | Object Count    | Share
-------------------------------------------------------
1     | US         |       16,503 |   50.3%
2     | CIS        |        6,706 |   20.4%
3     | PRC        |        5,880 |   17.9%
4     | UK         |          721 |    2.2%
5     | FR         |          649 |    2.0%
6     | JPN        |          315 |    1.0%
7     | TBD        |          208 |    0.6%
8     | IND        |          199 |    0.6%
9     | ITSO       |          138 |    0.4%
10    | ESA        |          127 |    0.4%
-------------------------------------------------------
✅ PASS: 129 unique owner codes verified as standardized.


### **Stage 4.4: Payload Vulnerability Report (Pre-Zombie Identification)**
**The Problem:** A satellite's operational status code only provides a snapshot of its current health, not its remaining utility. Payloads that are labeled as `OPERATIONAL` but have exceeded a 15-year design life represent "Next-Generation Zombies" — assets at high risk of sudden failure and loss of control.

**The Solution:** We execute a **preliminary vulnerability audit** by cross-referencing `ops_status` with `sat_age_years` using the industry-standard 15-year threshold. This provides a conservative baseline estimate prior to the high-fidelity Zombie identification in `03_orbital_risk_synthesis.ipynb`, which uses actual UCS design-life data.

**Note:** This is a SATCAT-only analysis. The final Zombie population will be calculated in the synthesis pipeline using the formula: `sat_age_years > (lifetime_years × 1.10)`.

In [16]:
# Define the Vulnerability Threshold (Industry Standard: 5 - 15 Years)
vulnerability_threshold = 15

# Isolate Operational Payloads
op_payloads_mask = (registry['in_orbit'] == 1) & \
                   (registry['object_type'] == 'PAYLOAD') & \
                   (registry['ops_status'] == 'OPERATIONAL')

op_payloads = registry[op_payloads_mask].copy()

# Calculate Vulnerability
total_op = len(op_payloads)
vulnerable_op = op_payloads[op_payloads['sat_age_years'] > vulnerability_threshold]
vulnerable_count = len(vulnerable_op)

print(f"{'--- PAYLOAD VULNERABILITY AUDIT ---':^55}")
print(f"Total Operational Payloads:     {total_op:,}")
print(f"Aged Assets (>15 Years):        {vulnerable_count:,}")
print(f"Vulnerability Rate:             {(vulnerable_count/total_op if total_op > 0 else 0):.1%}")
print("-" * 55)

# Breakdown by Orbit Class
if vulnerable_count > 0:
    print("\nVulnerability Distribution by Orbit:")
    print(vulnerable_op['orbit_class'].value_counts())

print(f"\n✅ Stage 4.4 Complete: {vulnerable_count} high-risk active assets identified.")

          --- PAYLOAD VULNERABILITY AUDIT ---          
Total Operational Payloads:     14,284
Aged Assets (>15 Years):        398
Vulnerability Rate:             2.8%
-------------------------------------------------------

Vulnerability Distribution by Orbit:
orbit_class
LEO           193
GEO           154
MEO            27
UNKNOWN        16
ELLIPTICAL      8
Name: count, dtype: int64

✅ Stage 4.4 Complete: 398 high-risk active assets identified.


### **Stage 5: Tiered Mass Imputation (The Proxy Engine)**
**The Problem:** Even after UCS enrichment, a ~82.9% gap remains, primarily composed of Debris and Rocket Bodies. To complete our kinetic model, these objects require a physical baseline.

**The Solution:** We initialize `proxy_mass_kg`. We preserve the high-fidelity UCS data where it exists, but fill the remaining `NaN` values using conservative averages from the **European Space Agency (ESA) Space Debris Report**.

In [17]:
# Initialize proxy_mass_kg with our enriched high-fidelity data
# This ensures that UCS-matched satellites keep their real mass.
registry['proxy_mass_kg'] = registry['launch_mass_kg']

# Object-Type-Specific proxies (ESA-aligned where possible)
# Based on ESA 2025 Space Environment Report when published, otherwise conservative assumptions
mass_proxies = {
    'ROCKET BODY': 2000.0,  # Assumed conservative; ESA 2025 does not publish a class-average
    'PAYLOAD': 355.0,       # ESA active payload mean mass (Sec. 3.5, Fig. 2.31)
    'DEBRIS': 50.0,         # Trackable fragment assumption; ESA does not provide a mean mass
    'UNKNOWN': 100.0        # Conservative fallback for unclassified objects
}

for category, mass_val in mass_proxies.items():
    # Only fill where mass is still missing after the UCS merge
    mask = (registry['object_type'] == category) & (registry['proxy_mass_kg'].isna())
    registry.loc[mask, 'proxy_mass_kg'] = mass_val

in_orbit_mask = registry['in_orbit'] == 1
total_mass_kg = registry[in_orbit_mask]['proxy_mass_kg'].sum()
total_mass_tonnes = total_mass_kg / 1000

print(f"\n{'--- MASS VALIDATION vs ESA 2025 ---':^50}")
print(f"Total In-Orbit Mass:     {total_mass_tonnes:,.0f} tonnes")
print("ESA 2025 Benchmark:      13,600-15,100 tonnes")
print("-" * 50)

# Breakdown by object type
mass_by_type = registry[in_orbit_mask].groupby('object_type')['proxy_mass_kg'].sum() / 1000
print("\nMass Distribution (Tonnes):")
for obj_type, mass in mass_by_type.sort_values(ascending=False).items():
    pct = (mass / total_mass_tonnes) * 100
    print(f"{obj_type:<15} {mass:>8,.0f}  ({pct:>5.1f}%)")



       --- MASS VALIDATION vs ESA 2025 ---        
Total In-Orbit Mass:     14,607 tonnes
ESA 2025 Benchmark:      13,600-15,100 tonnes
--------------------------------------------------

Mass Distribution (Tonnes):
PAYLOAD            9,135  ( 62.5%)
ROCKET BODY        4,799  ( 32.9%)
DEBRIS               668  (  4.6%)
UNKNOWN                5  (  0.0%)


### **Stage 5.1: Multi-Physics Proxy Imputation**
**The Problem:** While Mass is the primary gap, a significant portion of the debris population lacks observed **Inclination** or **RCS**. Without these, we cannot calculate orbital overlap or the cross-sectional area of a potential collision.

**The Solution:** We apply "Peer-Group" proxies for the remaining geometric gaps. 
* **Inclination:** We utilize the median inclination of the object's `orbit_class` to fill gaps.
* **RCS:** We apply categorical proxies based on `object_type` (e.g., Rocket Bodies are assigned a larger baseline signature than Debris fragments) to ensure 100% density across the kinetic profile.

In [18]:
# Define Physics Proxies for RCS (m^2)
rcs_proxies = {
    'ROCKET BODY': 5.0,   
    'PAYLOAD': 1.0,       
    'DEBRIS': 0.01,       
    'UNKNOWN': 0.01
}

print("Executing Multi-Physics Proxy Sweep...")

# Impute RCS
for category, rcs_val in rcs_proxies.items():
    mask = (registry['object_type'] == category) & (registry['rcs'].isna())
    registry.loc[mask, 'rcs'] = rcs_val

# Impute Inclination (Using Orbit Class Medians)
if registry['inclination_degrees'].isnull().any():
    inc_medians = registry.groupby('orbit_class')['inclination_degrees'].transform('median')
    registry['inclination_degrees'] = registry['inclination_degrees'].fillna(inc_medians)
    registry['inclination_degrees'] = registry['inclination_degrees'].fillna(registry['inclination_degrees'].median())

physics_gate = ['proxy_mass_kg', 'period_minutes', 'inclination_degrees', 'rcs', 'eccentricity']
in_orbit_mask = registry['in_orbit'] == 1

print(f"\n{'--- FINAL PHYSICS DENSITY GATE ---':^50}")
print(f"{'Attribute':<25} | {'Density'}")
print("-" * 50)

for col in physics_gate:
    density = registry[in_orbit_mask][col].notna().mean()
    print(f"{col:<25} | {density:>10.1%}")

print("-" * 50)
if registry[in_orbit_mask][physics_gate].isnull().sum().sum() == 0:
    print("🚀 PASS: All in-orbit assets are physics-complete.")

Executing Multi-Physics Proxy Sweep...

        --- FINAL PHYSICS DENSITY GATE ---        
Attribute                 | Density
--------------------------------------------------
proxy_mass_kg             |     100.0%
period_minutes            |     100.0%
inclination_degrees       |     100.0%
rcs                       |     100.0%
eccentricity              |     100.0%
--------------------------------------------------
🚀 PASS: All in-orbit assets are physics-complete.


### **Stage 5.2: Dry Mass Derivation & Fuel Fraction Audit**
**The Problem:** High-fidelity de-orbit modeling and collision analysis require a distinction between **Wet Mass** (Launch) and **Dry Mass** (End-of-Life). Using launch mass for inactive debris or exhausted rocket bodies overestimates the kinetic energy present in the orbital environment.

**The Solution:** We derive `dry_mass_kg` by applying industry-standard fuel fraction ratios.
* **Payloads:** 0.65 ratio for LEO (35% fuel); 0.55 for GEO/MEO/Elliptical (45% fuel for station-keeping).
* **Debris & Rocket Bodies:** 1.0 ratio (representing 100% structural dry mass).

**Note:** This differs from the UCS methodology (Stage 4.1), which uses observed median ratios from existing satellites. The SATCAT approach applies conservative industry standards to the broader debris population lacking empirical dry mass observations.

In [19]:
def calculate_dry_mass(row):
    """
    Applies orbit-specific mass ratios to derive dry mass from proxy_mass_kg.

    Update: Logic updated to match a more conservative fuel-fraction model
    based on recent literature (ESA 2025 Space Environment Report).
    """
    wet_mass = row['proxy_mass_kg']
    obj_type = row['object_type']
    regime = row['orbit_class']
    
    # Station-Keeping: The act of maintaining a satellite's orbit.
    # Typically requires small amounts of fuel for minor adjustments to keep
    # the satellite in its intended orbit. Satellites are affected by a variety
    # of external factors requiring periodic thruster adjustments.

    # Debris and Rocket Bodies are essentially dry structures (no fuel)
    if obj_type in ['DEBRIS', 'ROCKET BODY']:
        return wet_mass 
    
    # Payloads follow orbit-specific fuel-fraction logic
    if obj_type == 'PAYLOAD':
        if regime == 'LEO':
            # LEO satellites typically carry minimal fuel (<10%). They carried into orbit
            # by a launch vehicle and use small propulsion for station-keeping (minor delta-v 
            # thruster adjustments to keep the satellite from drifting out of its intended orbit).
            # Conservative estimate: 90% Dry Mass.
            return wet_mass * 0.90
        else:
            # GEO/MEO/Deep Space satellites often require significant fuel reserves
            # to reach and maintain their orbits, as well as for station-keeping maneuvers
            # Standard estimate: 55% Dry Mass.
            return wet_mass * 0.55
            
    return wet_mass

print("Deriving dry mass estimates...")
registry['dry_mass_kg'] = registry.apply(calculate_dry_mass, axis=1)

in_orbit_mask = registry['in_orbit'] == 1
total_wet = registry[in_orbit_mask]['proxy_mass_kg'].sum() / 1000
total_dry = registry[in_orbit_mask]['dry_mass_kg'].sum() / 1000

print(f"\n{'--- DRY MASS INTEGRITY AUDIT ---':^50}")
print(f"{'Metric':<25} | {'Value'}")
print("-" * 50)
print(f"{'Total Wet Mass (MT)':<25} | {total_wet:,.0f}")
print(f"{'Total Dry Mass (MT)':<25} | {total_dry:,.0f}")
print(f"{'Aggregate Fuel Load':<25} | {total_wet - total_dry:,.0f} MT")
print("-" * 50)

phys_errors = (registry[in_orbit_mask]['dry_mass_kg'] <= 0).sum()
if phys_errors == 0:
    print("🚀 PASS: Dry mass derived with 100% physical integrity.")

Deriving dry mass estimates...

         --- DRY MASS INTEGRITY AUDIT ---         
Metric                    | Value
--------------------------------------------------
Total Wet Mass (MT)       | 14,607
Total Dry Mass (MT)       | 12,562
Aggregate Fuel Load       | 2,045 MT
--------------------------------------------------
🚀 PASS: Dry mass derived with 100% physical integrity.


## **Cleaned SATCAT Registry: Data Dictionary**

#### **1. Physical & Kinetic Properties (Reconstructed)**
These columns represent the "Kinetic Engine" of the simulation. Gaps have been filled using high-fidelity UCS enrichment and **ESA-standard proxies**.

| Feature Name | Type | Description |
| :--- | :--- | :--- |
| `launch_mass_kg` | `float` | High-fidelity mass pulled from the **UCS Registry**. Remains `NaN` for debris. |
| `proxy_mass_kg` | `float` | **Final Kinetic Mass.** Uses high-fi UCS data or ESA categorical proxies. |
| `dry_mass_kg` | `float` | **Structural Mass.** Derived using orbit-specific Dry-to-Wet ratios (Stage 5.2). |
| `rcs` | `float` | Radar Cross Section ($m^2$). Sanitized and imputed via categorical baselines. |
| `rcs_class` | `str` | Derived size bin: **SMALL, MEDIUM, LARGE**. Based on numeric RCS. |
| `sat_age_years` | `Int64` | Calculated age ($2026 - launch\_year$) for fragmentation modeling. |

#### **2. Orbital Mechanics (Keplerian Verified)**
The geometry of the global tracked population. All entries are now physics-dense.

| Feature Name | Type | Description |
| :--- | :--- | :--- |
| `period_minutes` | `float` | Time for one orbit. **Derived via Kepler's 3rd Law** ($T = 2\pi\sqrt{a^3/\mu}$). |
| `inclination_degrees` | `float` | Angle relative to equator. Essential for collision probability maps. |
| `apogee_km` | `float` | Highest altitude point in orbit. |
| `perigee_km` | `float` | Lowest altitude point in orbit. |
| `eccentricity` | `float` | Deviation from circularity. **Mathematically derived** from altitudes. |
| `orbit_class` | `str` | Standardized regime: **LEO, MEO, GEO, Elliptical**. |

#### **3. Categorical & Temporal Metadata**
Standardized labels for relational integrity and geopolitical risk analysis.

| Feature Name | Type | Description |
| :--- | :--- | :--- |
| `norad_id` | `str` | **Primary Merge Key.** Normalized string for 1:1 join-readiness. |
| `cospar_id` | `str` | International designator (COSPAR ID). Scrubbed and capitalized. |
| `object_type` | `str` | **Controlled Vocabulary.** `PAYLOAD`, `ROCKET BODY`, `DEBRIS`, `UNKNOWN`. |
| `ops_status` | `str` | Human-readable health status (e.g., `OPERATIONAL`, `NON-OPERATIONAL`). |
| `owner_code` | `str` | ISO-style country/org code. Stripped of whitespace artifacts. |
| `launch_year` | `Int64` | **Derived Year of Launch.** Used for simulation aging. |
| `in_orbit` | `int` | Flag: `1` if currently active in orbit, `0` if decayed. |

### **Stage 6.1: Executive Pipeline Report**
**The Goal:** We generate a high-fidelity summary of the standardized registry. This report quantifies the "Density Gain" achieved through our physics reconstruction and identifies the core population of "Next-Generation Zombies" prior to the synthesis merge.

In [20]:
# Calculate Final Report Metrics
total_rows = len(registry)
in_orbit_mask = registry['in_orbit'] == 1
in_orbit_count = in_orbit_mask.sum()

# Count object types for the Composition Table
payload_count = (registry[in_orbit_mask]['object_type'] == 'PAYLOAD').sum()
rb_count = (registry[in_orbit_mask]['object_type'] == 'ROCKET BODY').sum()
debris_count = (registry[in_orbit_mask]['object_type'] == 'DEBRIS').sum()

# Mass Metrics (Wet vs Dry)
total_mass_mt = registry[in_orbit_mask]['proxy_mass_kg'].sum() / 1000
total_dry_mt = registry[in_orbit_mask]['dry_mass_kg'].sum() / 1000

# Health & Vulnerability Metrics
avg_age = registry[in_orbit_mask]['sat_age_years'].mean()
vulnerability_rate = (vulnerable_count / total_op) if total_op > 0 else 0

# Timeline Range
earliest_launch = int(registry['launch_year'].min())
latest_launch = int(registry['launch_year'].max())

# Define features for the Quality and Density Audit
physics_features = [
    'norad_id', 'cospar_id', 'object_type',            
    'period_minutes', 'inclination_degrees', 'rcs',    
    'proxy_mass_kg', 'dry_mass_kg', 'orbit_class',     
    'launch_year', 'sat_age_years', 'owner_code'       
]

# Generate Markdown Report
report = f"""
### **SATCAT Pipeline Completion Report**
**Total Global Registry:** {total_rows:,} Objects Tracked

#### **Tracking Timeline Summary**
| Metric | Value |
| :--- | :--- |
| **Oldest Catalog Entry** | {earliest_launch} |
| **Newest Catalog Entry** | {latest_launch} |
| **Operational History** | {latest_launch - earliest_launch} Years |

#### **Orbital Environment Health & Risk**
| Metric | Value | Note |
| :--- | :--- | :--- |
| **Average Object Age** | {avg_age:.1f} Years | Simulation Year: 2026 |
| **Total Kinetic Mass** | {total_mass_mt:,.0f} MT | High-Fi UCS + ESA Proxies |
| **Total Structural Mass**| {total_dry_mt:,.0f} MT | Derived Dry-Mass (Stage 5.2) |
| **Vulnerability Rate** | {vulnerability_rate:.1%} | Operational Payloads > 15 Years |

#### **Global Object Composition (In-Orbit)**
| Category | Count | Share |
| :--- | :--- | :--- |
| **Payloads** | {payload_count:,} | {payload_count/in_orbit_count:.1%} |
| **Rocket Bodies** | {rb_count:,} | {rb_count/in_orbit_count:.1%} |
| **Debris** | {debris_count:,} | {debris_count/in_orbit_count:.1%} |

#### **Data Quality and Density Engineering**
| Feature | Completeness | Method | Status |
| :--- | :--- | :--- | :--- |
"""


for feature in physics_features:
    coverage = registry[in_orbit_mask][feature].notna().mean()
    
    if feature in ['norad_id', 'cospar_id', 'owner_code']:
        method = "Sanitized/Verified"
    elif feature in ['period_minutes', 'sat_age_years', 'launch_year']:
        method = "Derived/Calculated"
    elif feature in ['object_type', 'orbit_class', 'rcs_class']:
        method = "Standardized/Mapped"
    elif feature in ['proxy_mass_kg', 'dry_mass_kg']:
        method = "UCS Enriched/Proxy"
    else:
        method = "Grouped Median"
        
    report += f"| **{feature.replace('_', ' ').title()}** | **{coverage:.1%}** | {method} | ✅ SUCCESS |\n"

display(Markdown(report))

output_path = '../data/clean/satcat_cleaned.csv'
registry.to_csv(output_path, index=False)
print(f"\n✅ EXPORT SUCCESS: {len(registry):,} records saved to {output_path}")


### **SATCAT Pipeline Completion Report**
**Total Global Registry:** 67,464 Objects Tracked

#### **Tracking Timeline Summary**
| Metric | Value |
| :--- | :--- |
| **Oldest Catalog Entry** | 1957 |
| **Newest Catalog Entry** | 2026 |
| **Operational History** | 69 Years |

#### **Orbital Environment Health & Risk**
| Metric | Value | Note |
| :--- | :--- | :--- |
| **Average Object Age** | 19.7 Years | Simulation Year: 2026 |
| **Total Kinetic Mass** | 14,607 MT | High-Fi UCS + ESA Proxies |
| **Total Structural Mass**| 12,562 MT | Derived Dry-Mass (Stage 5.2) |
| **Vulnerability Rate** | 2.8% | Operational Payloads > 15 Years |

#### **Global Object Composition (In-Orbit)**
| Category | Count | Share |
| :--- | :--- | :--- |
| **Payloads** | 17,730 | 54.0% |
| **Rocket Bodies** | 2,402 | 7.3% |
| **Debris** | 12,654 | 38.5% |

#### **Data Quality and Density Engineering**
| Feature | Completeness | Method | Status |
| :--- | :--- | :--- | :--- |
| **Norad Id** | **100.0%** | Sanitized/Verified | ✅ SUCCESS |
| **Cospar Id** | **100.0%** | Sanitized/Verified | ✅ SUCCESS |
| **Object Type** | **100.0%** | Standardized/Mapped | ✅ SUCCESS |
| **Period Minutes** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Inclination Degrees** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Rcs** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Proxy Mass Kg** | **100.0%** | UCS Enriched/Proxy | ✅ SUCCESS |
| **Dry Mass Kg** | **100.0%** | UCS Enriched/Proxy | ✅ SUCCESS |
| **Orbit Class** | **100.0%** | Standardized/Mapped | ✅ SUCCESS |
| **Launch Year** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Sat Age Years** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Owner Code** | **100.0%** | Sanitized/Verified | ✅ SUCCESS |



✅ EXPORT SUCCESS: 67,464 records saved to ../data/clean/satcat_cleaned.csv
